In [1]:
import findspark
findspark.init()
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StringType, StructType, TimestampType, IntegerType
import os
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'

In [2]:
# JSON Schema
impression_schema = StructType() \
    .add('ImpressionID', IntegerType()) \
    .add('UserID', IntegerType()) \
    .add('SessionID', IntegerType()) \
    .add('AdID', IntegerType()) \
    .add('AdCategory', StringType()) \
    .add('ImpressionTimestamp', TimestampType()) \
    .add('OS', StringType()) \
    .add('Browser', StringType()) \
    .add('Location', StringType()) \
    .add('Referrer', StringType()) \
    .add('ImpressionCost', FloatType()) \

click_schema = StructType() \
    .add('ImpressionID', IntegerType()) \
    .add('ClickTimestamp', TimestampType()) \
    .add('ClickID', StringType()) \
    .add('ClickPosition', StringType()) \
    .add('ClickCost', FloatType())

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .config("spark.jars", "postgresql-42.7.1.jar") \
    .appName("impression-click-event") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/23 17:17:00 WARN Utils: Your hostname, James-Razerblade resolves to a loopback address: 127.0.1.1; using 192.168.204.183 instead (on interface eth0)
24/06/23 17:17:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/limws/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/limws/.ivy2/cache
The jars for the packages stored in: /home/limws/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82b0c4d9-1be0-4df2-a434-7666c0eba43e;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-loggi

In [4]:
impression_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "impression-event") \
  .option("startingOffsets", "latest") \
  .load()

In [5]:
impression_json_df = impression_df.selectExpr("cast(value as string) as value")
impression_raw_df = impression_json_df.withColumn("value", from_json(impression_json_df["value"], impression_schema)).select("value.*")
impression_raw_df = impression_raw_df.replace("\"NaN\"", None)

In [6]:
click_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "click-event") \
  .option("startingOffsets", "latest") \
  .load()

In [7]:
click_json_df = click_df.selectExpr("cast(value as string) as value")
click_raw_df = click_json_df.withColumn("value", from_json(click_json_df["value"], click_schema)).select("value.*")
click_raw_df = click_raw_df.replace("\"NaN\"", None)

In [8]:
### apply watermarks on event-time columns
impressionsWithWatermark = impression_raw_df.withWatermark("ImpressionTimestamp", "1 minute").alias("impression")
clicksWithWatermark = click_raw_df.withWatermark("ClickTimestamp", "1 minute").alias("click")

### stream-stream inner join (no time constraints needed for inner join)
joined_stream = impressionsWithWatermark.join(
  clicksWithWatermark,
  expr("""
    click.ImpressionID = impression.ImpressionID AND
    click.ClickTimestamp >= impression.ImpressionTimestamp AND
    click.ClickTimestamp <= impression.ImpressionTimestamp + interval 1 minute
    """),
    "leftOuter"
)

In [9]:
joined_stream.writeStream \
    .format("console") \
    .start()

24/06/23 17:17:12 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-eb236722-12c7-4dec-88c6-b4c97c25f661. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/23 17:17:12 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [10]:
# conn = psycopg2.connect(
#     host="localhost",
#     database="superset",
#     user="superset",
#     password="superset")

# sql = """DROP TABLE IF EXISTS impression CASCADE"""
# cursor = conn.cursor()
# cursor.execute(sql)
# conn.commit()

In [11]:
# sql = """CREATE TABLE impression (
#     impressionid TEXT,
#     userid TEXT,
#     sessionid TEXT,
#     adid TEXT,
#     adcategory TEXT,
#     timestamp TIMESTAMP,
#     device TEXT,
#     os TEXT,
#     browser TEXT,
#     location TEXT,
#     pageid TEXT,
#     referrer TEXT,
#     impressioncost DECIMAL,
#     clickid TEXT NULL,
#     clickposition TEXT NULL,
#     clickcost DECIMAL NULL 
# );"""
# cursor = conn.cursor()
# cursor.execute(sql)
# conn.commit()

In [12]:
# def write_to_postgresql(df,epoch_id):
#     df.write \
#     .format('jdbc') \
#     .option("url", "jdbc:postgresql://localhost:5432/superset") \
#     .option("dbtable", "impression") \
#     .option("user", "superset") \
#     .option("password", "superset") \
#     .option("driver", "org.postgresql.Driver") \
#     .mode('append') \
#     .save()

In [13]:
# postgresql_stream=raw_df.writeStream \
#     .foreachBatch(write_to_postgresql) \
#     .start() \
#     .awaitTermination()

24/06/23 17:17:13 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/06/23 17:17:13 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/06/23 17:17:13 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/06/23 17:17:13 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
24/06/23 17:17:13 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
24/06/23 17:17:14 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/06/23 17:17:14 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/06/23 17:17:14 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/06/23 17:17:14 WARN AdminClientConfig: The c

-------------------------------------------
Batch: 0
-------------------------------------------
+------------+------+---------+----+----------+-------------------+---+-------+--------+--------+--------------+------------+--------------+-------+-------------+---------+
|ImpressionID|UserID|SessionID|AdID|AdCategory|ImpressionTimestamp| OS|Browser|Location|Referrer|ImpressionCost|ImpressionID|ClickTimestamp|ClickID|ClickPosition|ClickCost|
+------------+------+---------+----+----------+-------------------+---+-------+--------+--------+--------------+------------+--------------+-------+-------------+---------+
+------------+------+---------+----+----------+-------------------+---+-------+--------+--------+--------------+------------+--------------+-------+-------------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------+------+---------+----+----------+--------------------+-----+-------+--------+--------+--------------+------------+--------------------+-------+-------------+----------+
|ImpressionID|UserID|SessionID|AdID|AdCategory| ImpressionTimestamp|   OS|Browser|Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition| ClickCost|
+------------+------+---------+----+----------+--------------------+-----+-------+--------+--------+--------------+------------+--------------------+-------+-------------+----------+
|           6|    70|       80|  22| Education|2024-06-23 17:17:...|Linux|   Edge|   India| Twitter|     1.3441609|           6|2024-06-23 17:17:...|      6|       Header| 1.2847403|
|           5|    15|      135|  48|   Apparel|2024-06-23 17:17:...|macOS|Firefox|   China| Twitter|    0.02185926|           5|2024-06-23 17:17:...|      5|      Sidebar|

-------------------------------------------
Batch: 2
-------------------------------------------
+------------+------+---------+----+-------------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|ImpressionID|UserID|SessionID|AdID|   AdCategory| ImpressionTimestamp|     OS|Browser| Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition| ClickCost|
+------------+------+---------+----+-------------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|          20|     1|      106|   9|  Electronics|2024-06-23 17:17:...|    iOS|   Edge|   Canada|LinkedIn|    0.18715997|          20|2024-06-23 17:17:...|     20|        Popup| 1.9031632|
|          19|    12|       97|  25|      Apparel|2024-06-23 17:17:...|  macOS|   Edge|  Germany| Twitter|   0.045695744|          19|2024-06-23 17

-------------------------------------------
Batch: 3
-------------------------------------------
+------------+------+---------+----+----------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+----------+
|ImpressionID|UserID|SessionID|AdID|AdCategory| ImpressionTimestamp|     OS|Browser|Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition| ClickCost|
+------------+------+---------+----+----------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+----------+
|          22|    24|      135|  36|   Apparel|2024-06-23 17:17:...|Android|Firefox|     USA|  Google|     1.1074119|          22|2024-06-23 17:17:...|     22|       Banner| 1.7552674|
|          23|    93|      165|   2|  Services|2024-06-23 17:17:...|  Linux|Firefox|   China|  Google|     0.5467899|          23|2024-06-23 17:17:...|     23|    

...

-------------------------------------------
Batch: 17
-------------------------------------------
+------------+------+---------+----+-----------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|ImpressionID|UserID|SessionID|AdID| AdCategory| ImpressionTimestamp|     OS|Browser| Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition| ClickCost|
+------------+------+---------+----+-----------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|         115|    77|       88|  45|  Education|2024-06-23 17:18:...|  macOS|Firefox|    Japan|  Direct|      1.236103|        NULL|                NULL|   NULL|         NULL|      NULL|
|        NULL|  NULL|     NULL|NULL|       NULL|                NULL|   NULL|   NULL|     NULL|    NULL|          NULL|          93|2024-06-23 17:19:...| 

-------------------------------------------
Batch: 18
-------------------------------------------
+------------+------+---------+----+-------------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|ImpressionID|UserID|SessionID|AdID|   AdCategory| ImpressionTimestamp|     OS|Browser| Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition| ClickCost|
+------------+------+---------+----+-------------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+----------+
|         133|    31|       11|  32|    Education|2024-06-23 17:18:...|Android| Safari|      USA| Twitter|    0.02296535|        NULL|                NULL|   NULL|         NULL|      NULL|
|         126|    10|      186|   4|      Apparel|2024-06-23 17:18:...|  Linux|   Edge|    China|Facebook|     0.8664443|        NULL|            

-------------------------------------------
Batch: 19
-------------------------------------------
+------------+------+---------+----+-------------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+---------+
|ImpressionID|UserID|SessionID|AdID|   AdCategory| ImpressionTimestamp|     OS|Browser|Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition|ClickCost|
+------------+------+---------+----+-------------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+---------+
|         155|    17|      183|  17|     Services|2024-06-23 17:18:...|  macOS| Safari|     USA|  Direct|    0.72584283|        NULL|                NULL|   NULL|         NULL|     NULL|
|         157|    17|       20|   5|       Health|2024-06-23 17:18:...|Windows|   Edge|   Japan|  Google|    0.35645807|        NULL|                NULL|

-------------------------------------------
Batch: 20
-------------------------------------------
+------------+------+---------+----+-------------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+---------+
|ImpressionID|UserID|SessionID|AdID|   AdCategory| ImpressionTimestamp|     OS|Browser|Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition|ClickCost|
+------------+------+---------+----+-------------+--------------------+-------+-------+--------+--------+--------------+------------+--------------------+-------+-------------+---------+
|        NULL|  NULL|     NULL|NULL|         NULL|                NULL|   NULL|   NULL|    NULL|    NULL|          NULL|         115|2024-06-23 17:19:...|    115|       Header|0.1268125|
|         168|    75|      147|  37|    Education|2024-06-23 17:18:...|  macOS|   Edge|   Japan|  Direct|   0.053794757|        NULL|                NULL|

-------------------------------------------
Batch: 21
-------------------------------------------
+------------+------+---------+----+-----------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+---------+
|ImpressionID|UserID|SessionID|AdID| AdCategory| ImpressionTimestamp|     OS|Browser| Location|Referrer|ImpressionCost|ImpressionID|      ClickTimestamp|ClickID|ClickPosition|ClickCost|
+------------+------+---------+----+-----------+--------------------+-------+-------+---------+--------+--------------+------------+--------------------+-------+-------------+---------+
|         183|    53|      176|  39|  Education|2024-06-23 17:18:...|    iOS| Chrome|       UK|  Direct|     0.8720104|        NULL|                NULL|   NULL|         NULL|     NULL|
|         192|     9|       35|  24|     Health|2024-06-23 17:18:...|  Linux|   Edge|  Germany|  Direct|     1.0607641|        NULL|                NULL|   NU